In [2]:
%pip install tensorflow keras 


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install numpy pandas 


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re

In [5]:
datas = []
data_file_names = ['data_1.txt', 'data_2.txt']
for file_name in data_file_names:
    dfile = open(file_name, "r", encoding='utf8')
    data = dfile.read()
    datas.append(data)

In [6]:
for data in datas:
    print('Length of Book : ', len(data))

Length of Book :  1289729
Length of Book :  367437


In [7]:
def process_text(text):
    # Replace newlines with spaces
    text = text.replace('\n', ' ')

    # Remove special characters and multiple spaces
    processed_text = re.sub('[^a-zA-Z ]+', ' ', text)
    processed_text = re.sub(' +', ' ', processed_text)

    # Convert to lowercase and strip leading/trailing spaces
    processed_text = processed_text.lower().strip()

    return processed_text

In [8]:
for i in range(len(datas)):
    datas[i] = process_text(datas[i])
    print('Length of Book {}: {}'.format(i + 1, len(datas[i])))

Length of Book 1: 1224144
Length of Book 2: 349806


In [9]:
tokens = []
for i in range(len(datas)):
    tokens.append(datas[i].split())

In [10]:
for i in range(len(tokens)):
    print('Token Size : ', len(tokens[i]))

Token Size :  232058
Token Size :  63702


In [11]:
full_token = []
for i in range(len(tokens)):
    full_token = full_token + tokens[i]

In [12]:
len(full_token)
vocab = set(full_token)
len(vocab)


12789

In [13]:
length = 50 + 1
lines = []
for t in range(len(tokens)):
    for i in range(length, len(tokens[t])):
        seq = tokens[t][i-length : i]
        line = ' '.join(seq)
        lines.append(line)

In [14]:
len(lines)
lines[2]

'first monday of the month of april the market town of meung in which the author of romance of the rose was born appeared to be in as perfect a state of revolution as if the huguenots had just made a second la rochelle of it many citizens seeing the women'

In [15]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [16]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
sequences = np.array(sequences)
print(len(sequences))
print(sequences)

295658
[[  32    1  118 ...  356 4158  322]
 [   1  118 2909 ... 4158  322    1]
 [ 118 2909    3 ...  322    1  480]
 ...
 [   9  407  744 ...  159    1 1180]
 [ 407  744   33 ...    1 1180  523]
 [ 744   33   39 ... 1180  523    1]]


In [17]:
X, y = sequences[:, :-1], sequences[:, -1]

In [18]:
X[0]


array([   32,     1,   118,  2909,     3,     1,  1578,     3,  7897,
           1,  4856,   906,     3,  1511,     6,    23,     1, 12789,
           3,  3665,     3,     1,   629,    12,  2665,   198,     2,
          29,     6,    19,  1139,     5,   672,     3,  7895,    19,
          49,     1,  2168,    17,   219,    90,     5,   336,   274,
         640,     3,    13,   356,  4158])

In [19]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

12790


In [20]:
y = to_categorical(y, num_classes = vocab_size)

In [21]:
seq_length = X.shape[1]
seq_length

50

##MODEL

In [22]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(Bidirectional(LSTM(125, dropout=0.2)))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            639500    
                                                                 
 bidirectional (Bidirection  (None, 250)               176000    
 al)                                                             
                                                                 
 dense (Dense)               (None, 100)               25100     
                                                                 
 dense_1 (Dense)             (None, 12790)             1291790   
                                                                 
Total params: 2132390 (8.13 MB)
Trainable params: 2132390 (8.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
model.compile(loss ='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
history = model.fit(X, y, batch_size=128, epochs=30)

Epoch 1/30
2310/2310 [==============================] - 169s 73ms/step - loss: 6.3070 - accuracy: 0.0844
Epoch 2/30
2310/2310 [==============================] - 158s 68ms/step - loss: 5.6889 - accuracy: 0.1235
Epoch 3/30
2310/2310 [==============================] - 160s 69ms/step - loss: 5.4026 - accuracy: 0.1386
Epoch 4/30
2310/2310 [==============================] - 158s 68ms/step - loss: 5.1873 - accuracy: 0.1503
Epoch 5/30
2310/2310 [==============================] - 162s 70ms/step - loss: 5.0501 - accuracy: 0.1564
Epoch 6/30
2310/2310 [==============================] - 160s 69ms/step - loss: 4.9129 - accuracy: 0.1635
Epoch 7/30
2310/2310 [==============================] - 161s 70ms/step - loss: 4.8047 - accuracy: 0.1679
Epoch 8/30
2310/2310 [==============================] - 156s 68ms/step - loss: 4.7095 - accuracy: 0.1711
Epoch 9/30
2310/2310 [==============================] - 161s 70ms/step - loss: 4.6194 - accuracy: 0.1752
Epoch 10/30
2310/2310 [==============================] 

KeyboardInterrupt: 